# SGCN National List

The full [national list](https://www1.usgs.gov/csas/swap/national_list.html) of SGCN species across 2005 and 2015 represents a relatively complex query that needs to sum up the total states reporting each species. There may be some way to drive everything with some feature of the Elasticsearch index on the full original data that I haven't figured out yet, but I was only able to come up with a SQL statement to drive this.
```sql
 SELECT sgcn.scientificname_accepted AS scientificname,
    (array_agg(sgcn.taxonomicauthorityid_accepted ORDER BY sgcn.sgcnyear DESC))[1] AS taxonomicauthorityid,
    (array_agg(sgcn.commonname_submitted ORDER BY sgcn.sgcnyear DESC))[1] AS commonname,
    (array_agg(sgcn.taxonomicgroup_submitted ORDER BY sgcn.sgcnyear DESC))[1] AS taxonomicgroup,
    sum(((sgcn.sgcnyear = 2005))::integer) AS sgcn2005,
    sum(((sgcn.sgcnyear = 2015))::integer) AS sgcn2015
   FROM sgcn
  WHERE sgcn.taxonomicauthorityid_accepted <> ''
  GROUP BY sgcn.scientificname_accepted
```  
Running that live is way too costly on the system, so I built a view in GC2 using this select statement and indexed that in Elasticsearch as sgcn_nationallist. This results in a much more responsive query. This query selects only those records where there is an accepted taxonomic authority ID, which is the basic definition of what ends up on the national list.

In [1]:
import requests
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

In [2]:
#Class to render tables
class ListTable(list):
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

This query returns results from the Elasticsearch index for the sgcn_nationallist view. It only calls the first 100 results, so that will need to be paginated for the SWAP online app. I included the taxonomic authority ID as a reference. Those IDs to ITIS or WoRMS return a machine-readable response and are not content negotiable, so if we want to include them in the UI, we would need to translate the ID into something for humans.

In [3]:
sgcnNationalListURL = "https://gc2.mapcentia.com/api/v1/elasticsearch/search/bcb/public/sgcn_nationallist?size=25&from=25"
sgcnNationalList = requests.get(sgcnNationalListURL).json()

tableNationalList = ListTable()
tableNationalList.append(['Scientific Name', 'Common Name', '2005', '2015', 'Taxonomic Group', 'Taxonomic Authority ID/Link'])

for hit in sgcnNationalList['hits']['hits']:
    tableNationalList.append([hit['_source']['properties']['scientificname'], hit['_source']['properties']['commonname'], hit['_source']['properties']['sgcn2005'], hit['_source']['properties']['sgcn2015'], hit['_source']['properties']['taxonomicgroup'], hit['_source']['properties']['taxonomicauthorityid']])

display(tableNationalList)

Scientific Name,Common Name,2005,2015,Taxonomic Group,Taxonomic Authority ID/Link
Acroporidae,NA,0,1,Other Invertebrates,http://services.itis.gov/?q=tsn:52859
Actinemys marmorata,Western Pond Turtle,6,3,Reptiles,http://services.itis.gov/?q=tsn:668668
Adelocosa anops,Kauai cave wolf spider,1,0,Arachnids,http://services.itis.gov/?q=tsn:849993
Adenophorus periens,no common name,1,0,Plants,http://services.itis.gov/?q=tsn:17844
Adiantum aleuticum,Aleutian Maidenhair-fern,1,1,Plants,http://services.itis.gov/?q=tsn:181788
Aegialia concinna,Ciervo Aegialian Scarab Beetle,1,0,Insects,http://services.itis.gov/?q=tsn:926712
Aeropedellus clavatus,Club-horned Grasshopper,1,1,Insects,http://services.itis.gov/?q=tsn:657840
Aesculus glabra var. arguta,Ohio Buckeye,1,1,Plants,http://services.itis.gov/?q=tsn:28720
Aeshna interrupta,Variable Darner,1,3,Insects,http://services.itis.gov/?q=tsn:185979
Aethia cristatella,Crested Auklet,1,0,Birds,http://services.itis.gov/?q=tsn:177019


# Still to do

Included on the National List page in the current SWAP app are a couple of filtering features that allow users to select one or more years and/or taxonomic groups to view. These can likely be handled through aggregators in Elasticsearch. So far, I have not had any luck getting aggregator syntax to work through query DSL against the Elasticsearch API from GC2. I posted a question to the support guys to see if I can find some working examples. As far as I know, we should be able to easily configure aggs to work for both year (2005 or 2015) and the taxonomicgroup from the Elasticsearch index created from the database view.

I added size and from parameters to the above search example to demonstrate what can be used in paginating the national list results from Elasticsearch. These can also be submitted using Elasticsearch query DSL syntax instead of the discrete URL parameters.

The national list page also shows a "Progress Map" of states that have submitted lists for 2015. This is a simple query on distinct states from the sgcn table.